# This is Version 1 of our visualization tool
I use the following code to display feature lines (detected by CrestCODE based on curvature)

In [85]:
import os
import numpy as np
import open3d as o3d
import igl
import meshplot as mp

In [113]:
MESHNAME = "tbk_crop_230328_depth=12_correctNormals_crop_1_p1"
MESHFILE = "../Open3D_Test/" + MESHNAME + ".ply"
PSEUDO_PLY2 = 'ply2.txt'
NEWLINE = '\n'
NEIGHBOR = 6
CRESTLINE = 1 # 0 means NO Crest line, 1 means YES Crest line
TXT = ".txt"
RIDGES = "Ridge_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT
RAVINES = "Ravine_" + "k=" + str(NEIGHBOR) + "_" + MESHNAME + TXT
OUTPUT_rename = "Output_" + MESHNAME + TXT
PLY2_rename = "PLY2_" + MESHNAME + TXT
PLY = ".ply"

In [142]:
# read in mesh file
def get_mesh(path):
    mesh = o3d.io.read_triangle_mesh(MESHFILE)
    print(mesh)
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    return verts, faces

# turn meshfile to Pseudo_PLY2 file
def to_pseudo_PLY2(name, neighbor, crestline, verts, faces):
    NEWLINE = '\n'
    verts_count = len(verts)
    faces_count = len(faces)
    with open(name, 'w') as f:
        # num vertices
        f.write(str(verts_count) + NEWLINE)
        # num triangles
        f.write(str(faces_count) + NEWLINE)
        # neighborhood size
        f.write(str(neighbor) + NEWLINE)
        # Crestline Y/N
        f.write(str(crestline) + NEWLINE)
        # vertices (coordinates)
        for i in range(verts_count):
            f.write(" ".join(map(str, verts[i])) + NEWLINE)
        # faces (vertex IDs)
        for j in range(faces_count):
            f.write(" ".join(map(str, faces[j])) + NEWLINE)

# get data from Ridges.txt OR Ravines.txt
def RawData(FILE):
    f = open(FILE, 'r')
    V = int(f.readline()) # num of vertices
    E = int(f.readline()) # num of edges
    N = int(f.readline()) # num of connected components
    # vertices: [x, y, z, connected cmp ID]
    vertices = np.zeros(shape = (V, 4)) # index = vtx ID
    for i in range(V): 
        #index = pointID
        line = f.readline()
        vertices[i] = [float(n) for n in line.split()]
    # connected components: [Ridgeness, Sphericalness, Cyclideness]
    connected_cmp = np.zeros(shape = (N, 3)) # index = connected cmp ID
    for j in range(N):
        line = f.readline()
        connected_cmp[j] = [float(n) for n in line.split()]
    # edges (u,v): [vtx ID of u, vtx ID of v, triangle ID]
    edges = np.zeros(shape = (E, 3), dtype=int) # index = edge ID
    for k in range(E):
        line = f.readline()
        edges[k] = [n for n in line.split()]
    return V, N, E, vertices, connected_cmp, edges

# get vtx pair for Crest Line edges!
def DisplayData(V, N, E, vertices, connected_cmp, edges):
    # tuple: [V(0), N(1), E(2), vertices(3), connected components(4), edges(5)]
    Stt_Vtx = np.zeros(shape = (E, 3))
    End_Vtx = np.zeros(shape = (E, 3))
    for i in range(tuple[0]):
        Stt_Vtx[i] = vertices[3][edges[5][i, 0]][0:3]
        End_Vtx[i] = vertices[3][edges[5][i, 1]][0:3]
    Triangle_indices = edges[:,2]
    return Stt_Vtx, End_Vtx, Triangle_indices

def show(ridges, ravines, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RI_S, RI_T, RI_Tri = DisplayData(RawData(ridges))
    p.add_lines(RI_S, RI_T, shading={"line_color": "red", "line_width": 10.0})
    RA_S, RA_T, RA_Tri = DisplayData(RawData(ravines))
    p.add_lines(RA_S, RA_T, shading={"line_color": "blue", "line_width": 10.0})
    p.save("Ridge&Ravine_" + MESHNAME)

def ridges(ridges, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RI_S, RI_T, RI_Tri = DisplayData(RawData(ridges))
    p.add_lines(RI_S, RI_T, shading={"line_color": "red", "line_width": 10.0})
    p.save("Ridge_" + MESHNAME)

def ravines(ravines, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RA_S, RA_T, RA_Tri = DisplayData(RawData(ravines))
    p.add_lines(RA_S, RA_T, shading={"line_color": "blue", "line_width": 10.0})
    p.save("Ravine_" + MESHNAME)


In [148]:
def F():
    A = 1
    B = 2
    C = 3
    return (A, B, C)
def G(tuple={"A", "B", "C"}):
    D = 4
    return D
G(F())

4

In [137]:
verts, faces = get_mesh(MESHFILE)

TriangleMesh with 32104 points and 63500 triangles.


In [119]:
to_pseudo_PLY2(PSEUDO_PLY2, NEIGHBOR, CRESTLINE, verts, faces)

# BOOM! You ran CrestCODE and got your .txt outputs!
<br>You ran command in the CrestCODE folder (through Terminal) and got .txt files for: output, ravines, ridges
>the input: ./setCurvature ply2.txt output.txt
><br>Note that 'ply2.txt' corresponds to (the value of) PSEUDO_PLY2

<br>The .txt show up in the CrestCODE folder
<br>You moved those files Back to Open3D_Test!

In [114]:
#cleaning up!
os.rename("ridges.txt", RIDGES)
os.rename("ravines.txt", RAVINES)
os.rename("output.txt", OUTPUT_rename)
os.rename("ply2.txt", PLY2_rename)

In [218]:
verts, faces = get_mesh(MESHFILE)
# get data from Ridges.txt OR Ravines.txt
def RawData(FILE, vertices, faces):
    f = open(FILE, 'r')
    V = int(f.readline())
    E = int(f.readline())
    N = int(f.readline()) # num of connected components
    # vertices: [x, y, z, connected cmp ID]
    vertices = np.zeros(shape = (V, 4)) # index = vtx ID
    for i in range(V): 
        #index = pointID
        line = f.readline()
        vertices[i] = [float(n) for n in line.split()]
    # connected components: [Ridgeness, Sphericalness, Cyclideness]
    connected_cmp = np.zeros(shape = (N, 3)) # index = connected cmp ID
    for j in range(N):
        line = f.readline()
        connected_cmp[j] = [float(n) for n in line.split()]
    # edges (u,v): [vtx ID of u, vtx ID of v, triangle ID]
    edges = np.zeros(shape = (E, 3), dtype=int) # index = edge ID
    for k in range(E):
        line = f.readline()
        edges[k] = [n for n in line.split()]
    Edge_U = np.zeros(shape = (E, 3))
    Edge_V = np.zeros(shape = (E, 3))
    for i in range(E):
        Edge_U[i] = vertices[edges[i][0]][0:3]
        Edge_V[i] = vertices[edges[i][1]][0:3]
    Triangle_indices = edges[:,2]
    F = len(faces)  
    faces_bool = np.zeros(shape=(F,1))
    for e in range(E):
        if (edges[e][2] != -1):
            faces_bool[edges[e]] = 1
    return Edge_U, Edge_V, Triangle_indices, E, edges
start, finish, Triangle_indices, E, edges = RawData(RAVINES, verts, faces)
F = len(faces)
faces_bool = np.zeros(shape=(F,1))
for e in range(E):
    if (Triangle_indices[e] != -1):
        faces_bool[Triangle_indices[e]] = 1
print(faces_bool.shape)
print(np.sum(faces_bool, axis=0))
faces_to_show = []
for i in range(F):
    if(faces_bool[i] >= 1):
        faces_to_show.append(faces[i])
print(start[1])
print(finish[1])
p = mp.plot(v=verts, f=np.asarray(faces_to_show), return_plot=True)
p.add_lines(start, finish, shading={"line_color": "red", "line_width": 10.0})
p.save("TEST_Ravine_" + MESHNAME)

TriangleMesh with 32104 points and 63500 triangles.
(63500, 1)
[4867.]
[ 1.522417  2.35012  -0.171785]
[ 1.522818  2.350347 -0.171724]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.6551961…

Plot saved to file TEST_Ravine_tbk_crop_230328_depth=12_correctNormals_crop_1_p1.html.


In [200]:
"""#Example# get: faces; from: face index"""
F = np.asarray([0,1,2])
G = np.zeros(shape=(1,4))
G[:,0:3] = F
print(G)
# got: 
# [edge_u_index, edge_v_index, face_index] <- edges
# [a_index, b_index, c_index] @ face_index
# [x, y, z] @ a_index
# want:
# select faces by index
# tally appearance count for ea face

[[0. 1. 2. 0.]]


In [198]:
"""#Example# get: vertices; from: vertex index"""
A = np.asarray([[0, 1, 2]]) # edges, form: (u index, v index)
B = np.asarray([[5,6,0],[7,8,0],[9,10,0]]) # verts, form: (x, y, z)
# want: C = [5,6], D = [7,8] <- selected verts by indices; indices from edges
C = np.zeros(shape=(A.shape[0],2))
D = np.zeros(shape=(A.shape[0],2))
for i in range(len(A)):
    C[i] = B[A[i][0]][0:2]
    D[i] = B[A[i][1]][0:2]
print(B[1][0:2])
print(C)
print(D)

[7 8]
[[5. 6.]]
[[7. 8.]]


In [138]:
(V, N, E, CL_vertices, connected_cmp, edges) = RawData(RAVINES)
selected_faces = np.zeros((faces.shape[0],4))
selected_faces[:,0:3] = faces
selected_faces[1]



array([14., 13.,  3.,  0.])

In [135]:
triangle_indices = edges[:,2]
for i in range(len(triangle_indices)):
    if(i != -1):
        selected_faces[3] += 1
print(selected_faces[55498])

[28474. 28472. 24778.     0.]


In [143]:
# show ravines (very nice)
ravines(RAVINES, verts, faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.6551961…

TypeError: DisplayData() missing 5 required positional arguments: 'N', 'E', 'vertices', 'connected_cmp', and 'edges'

In [ ]:
# show entire mesh (no nothing)
p = mp.plot(verts, faces, return_plot=True)

In [52]:
# Save
p.save("FeatureLines_" + MESHNAME + "_neighbor=" + str(NEIGHBOR))

Plot saved to file FeatureLines_tbk_crop_230328_depth=12_correctNormals_crop_2_neighbor=6.html.


In [103]:
# Reference: 
# http://www.open3d.org/docs/0.11.1/tutorial/visualization/visualization.html
(V, N, E, vertices, connected_cmp, edges) = RawData(RAVINES)
points = vertices[:,:3]
lines = edges[:,:2]
colors = [[1, 0, 0] for i in range(E)]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_line_set(RAVINES + PLY, line_set, print_progress=True)
#test

Writing PLY: [========================================] 100%


True

In [ ]:
# all feature lines
show(RIDGES, RAVINES, verts, faces)
# just show ridges (not very useful)
ridges(RIDGES, verts, faces)